In [1]:
import pandas as pd

df = pd.read_csv("../data/data_raw.csv")

In [2]:
df.drop([df.columns[0], "CV"], axis=1, inplace=True)

In [3]:
df.columns = [
    "name",
    "job_name",
    "tags",
    "comment",
    "grade",
    "status",
    "final_status",
    "cv",
    "linkedin",
    "job_desc",
]

In [84]:
import pandas as pd

df_final = pd.read_csv("../data/data_final.csv")

In [85]:
df = df_final

In [86]:
df.drop(
    [
        "final_status_1",
        "final_status_2",
        "final_status_3",
        "status_1",
        "status_2",
        "status_3",
    ],
    axis=1,
    inplace=True,
)

In [87]:
df_split = df["final_status"].str.split(".", expand=True)

df_split.columns = ["final_status_1", "final_status_2", "final_status_3"]

df = pd.concat([df, df_split], axis=1)
df["final_status_1"] = df["final_status_1"].fillna("")
df["final_status_2"] = df["final_status_2"].fillna("")
df["final_status_3"] = df["final_status_3"].fillna("")

In [88]:
df_split = df["status"].str.split(".", expand=True)

df_split.columns = ["status_1", "status_2", "status_3"]

df = pd.concat([df, df_split], axis=1)

df["status_1"] = df["status_1"].fillna("")
df["status_2"] = df["status_2"].fillna("")
df["status_3"] = df["status_3"].fillna("")

In [ ]:
df.isna().sum()

In [90]:
def compute_overall_final_status(row):
    # Check if 'не рекомендуем' exists in the string value of 'final_status_3'
    if (
        isinstance(row["final_status_3"], str)
        and "не рекомендуем" in row["final_status_3"]
    ):
        return 0
    elif row["final_status_2"] == "5a":
        return 0
    elif row["final_status_1"] == "I" and row["final_status_2"] in ["1", "2", "3"]:
        return -1
    else:
        return 1


# Ensure that the DataFrame is not empty before applying the function
if not df.empty:
    df["overall_final_status"] = df.apply(compute_overall_final_status, axis=1)

In [91]:
def compute_overall_status(row):
    if (
        (isinstance(row["status_3"], str) and "не рекомендуем" in row["status_3"])
        or row["status_2"] == "5a"
        or row["status_1"] == "F"
    ):
        return 0
    elif (row["status_1"] == "I" and row["status_2"] in (["1", "2", "3"])) or row[
        "status_1"
    ] == "GG":
        return -1
    else:
        return 1


df["overall_status"] = df.apply(compute_overall_status, axis=1)

In [ ]:
df.columns

In [ ]:
df.groupby(
    ["final_status_1", "final_status_2", "final_status_3", "overall_final_status"]
).count().reset_index()

In [7]:
def convert_grade(grade):
    if isinstance(grade, str) and "/" in grade:
        numerator, denominator = map(int, grade.split("/"))
        return (numerator + denominator) / 2
    elif pd.notna(grade):
        return float(grade)
    else:
        return grade


df["grade"] = df["grade"].apply(convert_grade)

In [ ]:
df.groupby("grade").count().reset_index()

In [ ]:
df.isna().sum()

In [ ]:
df[df["cv"].isna() & df["linkedin"].isna()]

In [ ]:
df[df["cv"].isna()]

In [ ]:
import time

import pandas as pd
from linkedin_scraper import Person, actions
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run browser in headless mode
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)

linkedin_email = ""
linkedin_password = ""
actions.login(driver, linkedin_email, linkedin_password)


def get_text_resume(url):
    try:
        person = Person(url, driver=driver)
        time.sleep(3)  # Wait for the page to load
        return str(person) or None
    except Exception as e:
        return f"Error: {e}"


def fill_cv(row):
    if pd.notna(row["linkedin"]) and (pd.isna(row["cv"]) or row["cv"] == ""):
        return get_text_resume(row["linkedin"])
    return row["cv"]


tqdm.pandas()

df["cv"] = df.progress_apply(fill_cv, axis=1)


driver.quit()

In [ ]:
df.isna().sum()

In [96]:
df.to_csv("../data/data_final.csv", index=False)